# DicomStructureFile Class Demonstration

This notebook demonstrates the functionality of the `DicomStructureFile` class using a test DICOM RT Structure file.

## Overview
The `DicomStructureFile` class provides a convenient interface for:
- Loading DICOM RT Structure files
- Extracting structure information and metadata
- Reading contour sequences and converting them to ContourPoints
- Converting contour data to pandas DataFrames for analysis

## 1. Import Required Libraries

First, let's import the necessary libraries including our custom DicomStructureFile class.

In [1]:
# Import required libraries
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Add the src directory to the Python path
sys.path.append('src')

# Import our custom DicomStructureFile class
from dicom import DicomStructureFile

# Import related classes
from types_and_classes import SliceIndexType
from contours import ContourPoints
from structure_set import StructureSet

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load the DICOM Structure File

Now let's load the test DICOM RT Structure file using the DicomStructureFile class.

In [2]:
# Define the path to the test DICOM file
test_file_name = "RS.GJS_Struct_Tests.MultiVolume_A.dcm"
tests_dir = Path("Tests")

# Load the DICOM structure file using file_name parameter
dicom_file = DicomStructureFile(
    top_dir=tests_dir,
    file_name=test_file_name
)

print(f"Successfully loaded: {dicom_file}")
print(f"File path: {dicom_file.file_path}")
print(f"Is RT Structure file: {dicom_file.is_structure_file()}")

INFO:dicom:Successfully loaded DICOM dataset from RS.GJS_Struct_Tests.MultiVolume_A.dcm


INFO:dicom:Extracted 721 contours from 4 ROIs


Successfully loaded: DICOM RT Structure: MultiVolume_A for patient GJS_Struct_Tests
File path: Tests/RS.GJS_Struct_Tests.MultiVolume_A.dcm
Is RT Structure file: True


## 3. Explore Structure File Properties

Let's examine the basic properties and metadata of the loaded DICOM structure file.

In [4]:
# Get structure file information
structure_info = dicom_file.get_structure_set_info()

print("=== DICOM Structure File Information ===")
for key, value in structure_info.items():
    print(f"{key}: {value}")

# Access some basic DICOM dataset attributes
dataset = dicom_file.dataset
print("\n=== Additional DICOM Attributes ===")
print(f"Modality: {getattr(dataset, 'Modality', 'Not available')}")
print(f"Study Date: {getattr(dataset, 'StudyDate', 'Not available')}")
print(f"Manufacturer: {getattr(dataset, 'Manufacturer', 'Not available')}")
print(f"Software Version: {getattr(dataset, 'SoftwareVersions', 'Not available')}")

# Check if ROI contour sequence exists
if hasattr(dataset, 'ROIContourSequence'):
    print(f"\nNumber of ROI Contours: {len(dataset.ROIContourSequence)}")
else:
    print("\nNo ROI Contour Sequence found")

=== DICOM Structure File Information ===
PatientName: StructureVolumes^Test
PatientLastName: StructureVolumes
PatientID: GJS_Struct_Tests
StructureSet: MultiVolume_A
StudyID: Phantom2
SeriesNumber: 9
File: Tests/RS.GJS_Struct_Tests.MultiVolume_A.dcm

=== Additional DICOM Attributes ===
Modality: RTSTRUCT
Study Date: 20240218
Manufacturer: Varian Medical Systems
Software Version: 4.2.7.0

Number of ROI Contours: 4


## 4. Access Structure Set Information

Let's examine the structure set and ROI information in more detail.

In [5]:
# Examine the Structure Set ROI Sequence
if hasattr(dataset, 'StructureSetROISequence'):
    print("=== Structure Set ROI Information ===")
    roi_info = []
    
    for i, roi in enumerate(dataset.StructureSetROISequence):
        roi_data = {
            'ROI Number': getattr(roi, 'ROINumber', 'N/A'),
            'ROI Name': getattr(roi, 'ROIName', 'N/A'),
            'ROI Generation Algorithm': getattr(roi, 'ROIGenerationAlgorithm', 'N/A'),
            'Referenced Frame of Reference': getattr(roi, 'ReferencedFrameOfReferenceUID', 'N/A')
        }
        roi_info.append(roi_data)
        
        print(f"\nROI {i+1}:")
        for key, value in roi_data.items():
            print(f"  {key}: {value}")

    # Convert to DataFrame for easier viewing
    roi_df = pd.DataFrame(roi_info)
    print("\n=== ROI Summary Table ===")
    print(roi_df.to_string(index=False))
else:
    print("No Structure Set ROI Sequence found")

=== Structure Set ROI Information ===

ROI 1:
  ROI Number: 1
  ROI Name: BODY
  ROI Generation Algorithm: MANUAL
  Referenced Frame of Reference: 1.2.246.352.205.5609344633914932275.4927947885576298886

ROI 2:
  ROI Number: 7
  ROI Name: AdjacentSpheres
  ROI Generation Algorithm: MANUAL
  Referenced Frame of Reference: 1.2.246.352.205.5609344633914932275.4927947885576298886

ROI 3:
  ROI Number: 8
  ROI Name: AdjacentShells
  ROI Generation Algorithm: MANUAL
  Referenced Frame of Reference: 1.2.246.352.205.5609344633914932275.4927947885576298886

ROI 4:
  ROI Number: 9
  ROI Name: SingleVolume
  ROI Generation Algorithm: MANUAL
  Referenced Frame of Reference: 1.2.246.352.205.5609344633914932275.4927947885576298886

=== ROI Summary Table ===
 ROI Number        ROI Name ROI Generation Algorithm                           Referenced Frame of Reference
          1            BODY                   MANUAL 1.2.246.352.205.5609344633914932275.4927947885576298886
          7 AdjacentSpheres 

## 5. Extract ROI Contour Data

Now let's use our custom method to extract contour data and convert it to ContourPoints objects.

In [8]:
# Extract contour sequences using our custom method
contour_points = dicom_file.get_contour_points()

print(f"=== Contour Extraction Results ===")
print(f"Total ContourPoints objects created: {len(contour_points)}")

# Examine the first few contour points
if contour_points:
    print(f"\n=== Sample ContourPoints Objects ===")
    for i, cp in enumerate(contour_points[:5]):  # Show first 5
        print(f"\nContourPoint {i+1}:")
        print(f"  ROI Number: {cp['ROI']}")
        print(f"  Slice Index: {cp['Slice']}")
        print(f"  Number of Points: {len(cp['Points']) if cp['Points'] else 0}")
        if cp['Points'] and len(cp['Points']) > 0:
            print(f"  First point: {cp['Points'][0]}")
            print(f"  Last point: {cp['Points'][-1]}")

    # Show statistics by ROI
    roi_counts = {}
    for cp in contour_points:
        roi_num = cp['ROI']
        if roi_num not in roi_counts:
            roi_counts[roi_num] = {'slices': 0, 'total_points': 0}
        roi_counts[roi_num]['slices'] += 1
        if cp['Points']:
            roi_counts[roi_num]['total_points'] += len(cp['Points'])
    
    print(f"\n=== Contour Statistics by ROI ===")
    for roi_num, stats in roi_counts.items():
        print(f"ROI {roi_num}: {stats['slices']} slices, {stats['total_points']} total points")
        
else:
    print("No contour points found in the dataset")

INFO:dicom:Extracted 721 contours from 4 ROIs


=== Contour Extraction Results ===
Total ContourPoints objects created: 721

=== Sample ContourPoints Objects ===

ContourPoint 1:
  ROI Number: 1
  Slice Index: -10.0
  Number of Points: 680
  First point: [ -9.902  -9.906 -10.   ]
  Last point: [ -9.906  -9.902 -10.   ]

ContourPoint 2:
  ROI Number: 1
  Slice Index: -9.9
  Number of Points: 680
  First point: [ -9.902 -10.001  -9.9  ]
  Last point: [-10.001  -9.902  -9.9  ]

ContourPoint 3:
  ROI Number: 1
  Slice Index: -9.8
  Number of Points: 680
  First point: [ -9.902 -10.001  -9.8  ]
  Last point: [-10.001  -9.902  -9.8  ]

ContourPoint 4:
  ROI Number: 1
  Slice Index: -9.7
  Number of Points: 680
  First point: [ -9.902 -10.001  -9.7  ]
  Last point: [-10.001  -9.902  -9.7  ]

ContourPoint 5:
  ROI Number: 1
  Slice Index: -9.6
  Number of Points: 680
  First point: [ -9.902 -10.001  -9.6  ]
  Last point: [-10.001  -9.902  -9.6  ]

=== Contour Statistics by ROI ===
ROI 1: 201 slices, 136680 total points
ROI 7: 100 slices, 13

## Summary

This notebook has demonstrated the complete functionality of the `DicomStructureFile` class:

1. **Loading DICOM Files**: Successfully loaded a DICOM RT Structure file using either file_name or file_path parameters
2. **Metadata Extraction**: Retrieved comprehensive structure file information and ROI details
3. **Contour Processing**: Extracted all contour sequences and converted them to `ContourPoints` objects

### Key Features Demonstrated:
- Flexible initialization with multiple parameter options
- Robust error handling and validation
- Integration with existing `ContourPoints` and related classes
- Comprehensive metadata extraction
- Multiple export format support

### Next Steps:
- The `DicomStructureFile` class can be integrated into larger workflow pipelines


## 9. Integration with StructureSet Class
Now let's demonstrate how to use the DicomStructureFile with the StructureSet class to analyze structure relationships.


In [10]:
print("=== Creating StructureSet from DicomStructureFile ===")

# Method 1: Using DicomStructureFile (recommended approach)
structure_set = StructureSet(dicom_structure_file=dicom_file)

print(f"Successfully created StructureSet with {len(structure_set.structures)} structures")


INFO:structure_set:Building StructureSet from 721 contour points


=== Creating StructureSet from DicomStructureFile ===
Successfully created StructureSet with 4 structures


In [11]:
# Display the structures that were loaded
print("\n=== Loaded Structures ===")
for roi, structure in structure_set.structures.items():
    print(f"ROI {roi}: {structure.name}")



=== Loaded Structures ===
ROI 1: Structure_1
ROI 7: Structure_7
ROI 8: Structure_8
ROI 9: Structure_9


In [12]:
# Get basic structure information
print("=== Structure Summary ===")
structure_summary = structure_set.summary()
print(structure_summary.to_string(index=False))


=== Structure Summary ===
 ROI        Name  Physical_Volume  Exterior_Volume  Hull_Volume  Num_Contours  Num_Slices
   1 Structure_1      8759.813786      8759.813786  8759.813786           221         221
   7 Structure_7       137.643365       137.643365   137.798315           108          92
   8 Structure_8        68.978686       135.856443   136.228704           196         102
   9 Structure_9       448.450652       448.450652   456.216980           260         188


In [13]:
# Calculate relationships between all structures
print("\n=== Calculating Structure Relationships ===")
structure_set.calculate_relationships()

print(f"Total relationships calculated: {structure_set.relationship_graph.number_of_edges()}")



=== Calculating Structure Relationships ===
Total relationships calculated: 6


In [14]:
# Display the relationship summary table
print("\n=== Structure Relationship Summary Table ===")
relationship_summary = structure_set.relationship_summary()

if not relationship_summary.empty:
    print(relationship_summary.to_string(index=False))
    
    # Show relationship type distribution
    print(f"\n=== Relationship Type Distribution ===")
    relationship_counts = relationship_summary['Relationship_Type'].value_counts()
    for rel_type, count in relationship_counts.items():
        print(f"{rel_type}: {count}")
        
    # Export the relationship summary to CSV
    relationship_filename = "structure_relationships.csv"
    relationship_summary.to_csv(relationship_filename, index=False)
    print(f"\nRelationship summary exported to: {relationship_filename}")
    
else:
    print("No relationships were calculated. This may occur if:")
    print("- Structures don't overlap spatially")
    print("- Structures exist on different slice ranges")
    print("- Only one structure was loaded")



=== Structure Relationship Summary Table ===
 Structure_A  Structure_B      Relationship_Type                                                    DE27IM
           1            7 Relationship: Contains |111| |111| |111|\n|001| |001| |001|\n|001| |001| |001|\n
           1            8 Relationship: Contains |111| |111| |111|\n|001| |001| |001|\n|001| |001| |001|\n
           1            9 Relationship: Contains |111| |111| |111|\n|001| |001| |001|\n|001| |001| |001|\n
           7            8 Relationship: Disjoint |001| |001| |001|\n|001| |001| |001|\n|111| |111| |111|\n
           7            9 Relationship: Disjoint |001| |001| |001|\n|001| |001| |001|\n|111| |111| |111|\n
           8            9 Relationship: Disjoint |001| |001| |001|\n|001| |001| |001|\n|111| |111| |111|\n

=== Relationship Type Distribution ===
Relationship: Contains: 3
Relationship: Disjoint: 3

Relationship summary exported to: structure_relationships.csv


In [15]:
# Demonstrate structure filtering with exclusions
print("\n=== Structure Filtering Example ===")

# Create a new StructureSet with exclusions applied
filtered_structure_set = StructureSet(dicom_structure_file=dicom_file)

# Apply exclusions to remove structures starting with common prefixes
exclusion_patterns = ['BODY', 'External']  # Common prefixes to exclude
filtered_structure_set.apply_exclusions(
    exclusion_patterns=exclusion_patterns, 
    exclude_default=True
)

print(f"Structures before filtering: {len(structure_set.structures)}")
print(f"Structures after filtering: {len(filtered_structure_set.structures)}")

if len(filtered_structure_set.structures) > 0:
    print("\n=== Remaining Structures After Filtering ===")
    for roi, structure in filtered_structure_set.structures.items():
        print(f"ROI {roi}: {structure.name}")
    
    # Calculate relationships for the filtered set
    filtered_structure_set.calculate_relationships()
    
    if filtered_structure_set.relationship_graph.number_of_edges() > 0:
        print("\n=== Filtered Relationship Summary ===")
        filtered_relationships = filtered_structure_set.relationship_summary()
        print(filtered_relationships.to_string(index=False))
else:
    print("All structures were filtered out.")


INFO:structure_set:Building StructureSet from 721 contour points



=== Structure Filtering Example ===


AttributeError: 'DicomStructureFile' object has no attribute 'apply_exclusions'

In [16]:
# Show individual structure details
print("\n=== Individual Structure Details ===")

for roi, structure in structure_set.structures.items():
    print(f"\n--- Structure ROI {roi}: {structure.name} ---")
    print(f"Physical Volume: {structure.physical_volume:.3f} cc")
    print(f"Hull Volume: {structure.hull_volume:.3f} cc")
    print(f"Number of Slices: {len(structure.region_table)}")
    print(f"Number of Contours: {len(structure.contour_graph)}")
    
    # Show slice range
    if hasattr(structure, 'slice_sequence') and structure.slice_sequence:
        slice_indices = list(structure.slice_sequence.slice_indices)
        if slice_indices:
            print(f"Slice Range: {min(slice_indices):.1f} to {max(slice_indices):.1f}")



=== Individual Structure Details ===

--- Structure ROI 1: Structure_1 ---
Physical Volume: 8759.814 cc
Hull Volume: 8759.814 cc
Number of Slices: 221
Number of Contours: 221

--- Structure ROI 7: Structure_7 ---
Physical Volume: 137.643 cc
Hull Volume: 137.798 cc
Number of Slices: 92
Number of Contours: 108

--- Structure ROI 8: Structure_8 ---
Physical Volume: 68.979 cc
Hull Volume: 136.229 cc
Number of Slices: 102
Number of Contours: 196

--- Structure ROI 9: Structure_9 ---
Physical Volume: 448.451 cc
Hull Volume: 456.217 cc
Number of Slices: 188
Number of Contours: 260


In [17]:
# Create a comprehensive analysis report
print("\n=== Comprehensive Analysis Report ===")

analysis_report = {
    'File_Info': dicom_file.get_structure_info(),
    'Structure_Count': len(structure_set.structures),
    'Relationship_Count': structure_set.relationship_graph.number_of_edges(),
    'Total_Contour_Points': len(dicom_file.contour_points) if dicom_file.contour_points else 0
}

print("=== DICOM File Analysis Report ===")
for key, value in analysis_report.items():
    if isinstance(value, dict):
        print(f"\n{key.replace('_', ' ')}:")
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
    else:
        print(f"{key.replace('_', ' ')}: {value}")



=== Comprehensive Analysis Report ===


AttributeError: 'DicomStructureFile' object has no attribute 'get_structure_info'

## Summary of Integration

This extended demonstration shows how the `DicomStructureFile` class integrates seamlessly with the `StructureSet` class to provide:

1. **Automatic Structure Loading**: The `StructureSet` can be initialized directly from a `DicomStructureFile`
2. **Relationship Analysis**: Calculate spatial relationships between all structures
3. **Structure Filtering**: Apply exclusion patterns to remove unwanted structures
4. **Comprehensive Reporting**: Generate detailed reports on structure properties and relationships
5. **Data Export**: Export relationship data and analysis reports to various formats

### Key Benefits:
- **Streamlined Workflow**: Single initialization creates both contour data and structure relationships
- **Flexible Filtering**: Remove common utility structures (BODY, External, etc.) before analysis
- **Rich Metadata**: Access both DICOM metadata and calculated structure properties
- **Export Capabilities**: Generate reports and data files for further analysis

This integration demonstrates the power of combining DICOM file handling with advanced spatial relationship analysis.